In [1]:
import cv2
import numpy as np
import pandas as pd

import pytesseract
from pytesseract import Output

import re

import skimage.filters as filters
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import PIL.Image

import os, random, pathlib, warnings, itertools, math, argparse, imutils
warnings.filterwarnings("ignore")

import import_ipynb

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'E:\Programmes\Tesseract-OCR\tesseract.exe'
tessdata_dir_config = r'E:\Programmes\Tesseract-OCR\tesseract.exe\tessdata' 
tessdata_dir_config1 = r'E:\Programmes\Tesseract-OCR\tesseract.exe\tessdata_fast' 
tessdata_dir_config2 = r'E:\Programmes\Tesseract-OCR\tesseract.exe\tessdata_best' 
custom_config0 = r'--oem 3 --psm 6'
custom_config1 = r'--oem 3 --psm 11'
custom_config2 = r'--oem 3 --psm 8'
custom_config3 = r'--oem 1 --psm 3'

In [3]:
liste = [tessdata_dir_config, tessdata_dir_config1, tessdata_dir_config2, custom_config0, custom_config1, custom_config2, custom_config3]

In [4]:
def sketch_transform(image):  # in here you can do image filteration
    image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_grayscale_blurred = cv2.GaussianBlur(image_grayscale, (7,7), 0)
    image_canny = cv2.Canny(image_grayscale_blurred, 10, 80)
    _, mask = image_canny_inverted = cv2.threshold(image_canny, 30, 255, cv2.THRESH_BINARY_INV)
    return mask

def sketch_transform1(img):
    mask = np.zeros(img.shape, dtype=np.uint8)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    smooth = cv2.GaussianBlur(gray, (95,95), 0)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    division = cv2.divide(gray, smooth, scale=255)
    sharp = filters.unsharp_mask(division, radius=1.5, amount=1.5, channel_axis=False, preserve_range=False)
    sharp = (255*sharp).clip(0,255).astype(np.uint8)
    thresh = cv2.threshold(sharp, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return thresh

def tessdata(img, par):
    #d = pytesseract.image_to_data(img)
    d = pytesseract.image_to_data(img, lang='fra', config=par, output_type=Output.DICT) #thresh
    return d

def tessdata1(img, par):
    text2 = pytesseract.image_to_string(img, lang='fra', config=par)  #mask or thresh 
    #print(text2) #thresh
    #return text2
    nums = re.findall(r'[A-Z][0-9][0-9][0-9][0-9][0-9][0-9].|[A-Z][0-9][0-9][0-9][0-9][0-9][0-9][0-9]', text2)
    return nums

def spacebar(list):
    L = []
    for i in list:
        j = i.replace(' ','')
        L.append(j)
    return L

def extraction(img1, img2, par1, par2):
    d = tessdata(img1, par1)
    #print(d)
    nums = tessdata1(img2, par2)
    L = spacebar(nums)
    #print(L)
    #f = L[0]
    #text = None
    #conf = 0
    #M = 0,0,0,0
    return d, L

def box(img, conf, text, par):
    x, y, w, h = par
    text = "".join(text).strip()
    cv2.rectangle(img,
                (x, y),
                (x + w, y + h),
                (0, 0, 255), 2)
    cv2.putText(img,
                text,
                (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                1.2, (0, 255, 255), 3)
    cv2.imshow("test", img)
    k = cv2.waitKey(1)
    return("Confidence: {}".format(conf), "Text: {}".format(text))

def Localise(img1, img2):
    for a in liste:
        for b in liste : 
            d, L = extraction(img1, img2,a,b)
            print(L)
            if L !=[] :
                for i in range(0, len(d["text"])):
                    x = d["left"][i]
                    y = d["top"][i]
                    w = d["width"][i]
                    h = d["height"][i]
                    text = d["text"][i]
                    conf = int(float(d["conf"][i]))
                    M = x, y, w, h
                    if str(L[0]) == str(text) and (conf is not None) and (text is not None) and (M is not None) :
                        return (conf, text, M)                            
                        break
                    else :
                        continue
                    break
            else :
                continue

In [ ]:
videoCaptureObject = cv2.VideoCapture(0)
 
upper_left = (50, 50)
bottom_right = (500, 300)

result = True
while(result):
    ret,image_frame = videoCaptureObject.read()

    #Rectangle marker
    r = cv2.rectangle(image_frame, upper_left, bottom_right, (100, 50, 200), 5)
    rect_img = image_frame[upper_left[1] : bottom_right[1], upper_left[0] : bottom_right[0]]
    
    sketcher_rect = rect_img
    
    cv2.imshow("test", image_frame)
    cv2.waitKey(1)
    
    if cv2.waitKey(1) & 0xFF == ord('y'): #save on pressing 'y' 
        #sketcher_rect = sketch_transform1(sketcher_rect)
        conf, text, M = Localise(sketcher_rect, sketcher_rect)
        box(sketcher_rect, conf, text, M)
        
    
    #Conversion for 3 channels to put back on original image (streaming)
    #sketcher_rect_rgb = cv2.cvtColor(sketcher_rect, cv2.COLOR_GRAY2RGB)
    
    #Replacing the sketched image on Region of Interest
    #image_frame[upper_left[1] : bottom_right[1], upper_left[0] : bottom_right[0]] = sketcher_rect
    
    
 
videoCaptureObject.release()
cv2.destroyAllWindows()